In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following.npy')
image_2048 = np.load('../Data/npy/Social_64D.npy')
user_category = np.load('../Data/npy/user_category_1216.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')

In [4]:
print('user_following shape ',user_following.shape)
print('following_64 shape ',image_2048.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (1489, 88)
following_64 shape  (88, 64)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)


In [5]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (1489, 17)


In [6]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
print(following_true)

[[13, 15, 16, 24, 29, 37, 44], [3, 25, 67, 71, 74, 75, 82], [8, 11, 24, 28, 41, 63, 67, 79, 86], [8, 24, 28, 29, 63, 70, 79], [8, 24, 37, 63, 67, 70, 79], [8, 17, 24, 25, 31, 43, 63, 67, 70, 73, 74, 79], [3, 10, 11, 22, 40, 44, 47, 74, 76, 82], [8, 17, 21, 27, 28, 32, 33, 45, 49, 67, 75, 82, 84], [3, 25, 63, 74, 79], [3, 10, 11, 14, 25, 40, 45, 54, 60, 65, 67, 74, 75, 79, 82], [3, 11, 22, 24, 40, 54, 57, 63, 67, 74, 76, 79], [7, 11, 14, 24, 25, 40, 43, 54, 63, 67, 75, 79], [7, 8, 24, 41, 63, 79], [0, 11, 12, 25, 26, 27, 32, 33, 42, 44, 45, 60, 67, 69, 71, 72, 74, 76, 79, 82, 84], [8, 24, 29, 43, 57, 63, 67, 76, 79], [3, 8, 25, 28, 40, 75, 79], [8, 54, 63, 67, 70, 79], [3, 9, 10, 25, 40, 44, 45, 54, 60, 67, 71, 74, 76, 82, 84], [8, 10, 28, 40, 43, 45, 54, 67, 76, 79], [25, 32, 33, 67, 71, 82, 83, 84], [8, 24, 43, 63, 67, 70, 79], [8, 24, 28, 29, 41, 44, 57, 79], [8, 14, 24, 29, 41, 43, 57, 63, 67, 78, 79], [0, 12, 26, 27, 32, 33, 53, 55, 72, 84], [12, 17, 18, 27, 32, 33, 43, 45, 56, 63,

In [7]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  34


# Training data and Testing data

In [8]:
test_amount = 150

In [9]:
user_idx = [i for i in range(len(user_following))]
#test_idx is the number of user for testing
test_idx = random.sample(user_idx,test_amount)

In [10]:
# Training  and Testing
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        # random choose 2 true and 8 false for test 
        t_for_test = random.sample(temp_t,2)
        f_for_test  = random.sample(temp_f,8)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [11]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1489
The length of train_f: 1489
The length of test_t: 150
The length of test_f: 150


In [38]:
user_aux_size = [len(train_t[i]) for i in range(len(train_t))]
print(user_aux_size)

[7, 7, 9, 7, 7, 12, 10, 13, 5, 15, 12, 12, 6, 21, 9, 7, 4, 15, 10, 8, 7, 6, 11, 10, 14, 7, 11, 10, 12, 11, 14, 5, 10, 17, 6, 6, 6, 11, 5, 3, 20, 6, 10, 10, 16, 12, 11, 11, 5, 6, 11, 7, 11, 5, 13, 7, 11, 8, 11, 8, 5, 19, 10, 3, 10, 16, 11, 19, 6, 6, 9, 15, 11, 9, 7, 11, 5, 10, 9, 10, 9, 12, 7, 6, 13, 12, 4, 10, 10, 9, 12, 13, 7, 6, 11, 19, 13, 12, 11, 7, 7, 17, 13, 5, 12, 7, 12, 11, 9, 8, 6, 7, 12, 11, 8, 9, 13, 14, 11, 16, 6, 11, 5, 17, 5, 12, 5, 15, 10, 10, 18, 15, 13, 5, 5, 5, 8, 8, 6, 7, 12, 14, 13, 7, 20, 6, 5, 12, 13, 5, 10, 5, 9, 12, 18, 12, 11, 7, 11, 7, 12, 7, 16, 18, 7, 5, 6, 10, 7, 10, 7, 11, 5, 10, 10, 6, 11, 9, 19, 21, 5, 6, 11, 6, 10, 9, 7, 13, 18, 8, 12, 6, 8, 14, 12, 16, 8, 7, 9, 24, 14, 13, 11, 8, 5, 13, 10, 9, 12, 11, 8, 6, 19, 13, 6, 12, 12, 7, 23, 5, 10, 6, 6, 18, 14, 10, 18, 15, 5, 21, 10, 5, 7, 7, 10, 14, 9, 22, 11, 10, 8, 15, 19, 8, 13, 12, 9, 10, 5, 7, 11, 11, 13, 9, 12, 7, 12, 5, 7, 9, 10, 13, 12, 5, 16, 6, 7, 11, 8, 15, 6, 4, 6, 5, 6, 10, 12, 12, 5, 11, 6, 11, 

# Recommendation  Module

In [13]:
"""
n: the number of users
m: the number of YouTubers
k: latent dims
l: feature dims
"""
n = 1489 
m = 88  
k = 128
l = 64

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None,l))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))


image_i = tf.placeholder(tf.float32, shape=(1,l))
image_j = tf.placeholder(tf.float32, shape=(1,l))

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [n, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    W1 = tf.get_variable("W1", [n, k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [k,k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [m,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [k,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
    
    

    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    Beta = tf.get_variable("beta", [n,l],
                             # initializer=tf.contrib.layers.xavier_initializer())
                                     initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i) 
vj = tf.nn.embedding_lookup(item_latent, j)

w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
wu = Wu
#wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
wa = Wa
#wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(k*k)
wv = Wv
#wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [14]:
a_list=tf.Variable([])
q = tf.constant(0)
def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])
def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    
    a_list = tf.concat([a_list,[(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
        tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q]]],0)
    q += 1
    return q,  a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

test = tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[0]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[0]),0), transpose_b=True) +
        tf.matmul(wv, tf.expand_dims(xf[0],0), transpose_b=True)))
"""
for q in range(3): #取l_id個 YouTuber 的 類別
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    a_list.append((tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
        tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q])
"""
a_list_soft=tf.nn.softmax(a_list)


aux_np = tf.expand_dims(tf.zeros(128),0) #dimension (1,32)
q = tf.constant(0)
def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)])  # [[7, 16], [10, 25]]
    q += 1
    return q, aux_np

_,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

"""
for q in range(3): #取q個auxliary item
    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
"""


aux_np+=u #user_latent factor + sum (alpha*auxilary)
aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np

#矩陣中對應函數各自相乘
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,image_i, transpose_b=True)
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,image_j, transpose_b=True)

xuij = xui- xuj

l2_norm = tf.add_n([
            0.001 * tf.reduce_sum(tf.multiply(u, u)),
            0.001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.001 * tf.reduce_sum(tf.multiply(w1, w1)),
            0.001 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.001 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.001 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.001 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.1 * tf.reduce_sum(tf.multiply(beta,beta)),
            
          ])

loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

Instructions for updating:
Use `tf.cast` instead.


In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0=time.time()

#use_true=init_list_of_objects(136)
#use_test=init_list_of_objects(136)

train_pair_t=[] #positive feedback
train_pair_f=[] #negative feedback
train_yes_id=[] 
for q in range(10):
    print('Iteraction:',q)
    train_auc=0
    total_loss=0
    xuij_auc=0
    length = 0
    for z in range(1489):
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes=[]
        yesr=[]
        r_3=np.zeros(len(train_t[z])) 
        
        #這裡不知道怎麼讓3變成變動型的長度
        sample=random.sample(train_t[z],len(train_t[z])) #隨機選3個sample true's YouTuber
        train_yes_id.append(sample) #sample全部丟進去
        
        for k in range(len(sample)):
            yes.append(image_2048[sample[k]])
            yesr.append(YouTuber_category[sample[k]]*user_category_norm[z])
            #print('YouTuber_category ', YouTuber_category[sample[k]])
            #print('User_category ',user_category_norm[z])
        for k in range(len(sample)):
            r_3[k]=max(yesr[k])
        #print('r_3:',r_3)
        
        yes=np.array(yes)
        #print('user shape should be ',np.array([z]).shape)
        #print('xf shape should be ',yes.shape)
        #print('r shape should be ',np.array(r_3).shape)
        #print('l_id shape should be ',np.array(sample).shape)
        
        #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
        train_t_sample = random.sample(train_t[z],len(train_t[z]))
        #print('number of positive feedback', len(train_t[z]))
        for ta in train_t_sample:
            #ta=random.choice(train_t[z]) #ta is true positve photo
            train_pair_t.append(ta)
            image_1=np.expand_dims(image_2048[ta],0) #(1,2048)
            #print('Image_1 shape ',image_1.shape)
            train_f_sample = random.sample(train_f[z],20)
            for b in train_f_sample:
                #print('likes:',ta,';Not likes:',b)
                #b=random.choice(train_f[z])  #b is no feedback photo
                train_pair_f.append(b)
                image_2=np.expand_dims(image_2048[b],0) #(1,2048)
                #print('Image_2 shape',image_2.shape)
            
                #use_test[z].append(b)
                _a_list,r3,_auc, _loss,_=sess.run([a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: yes , l_id:sample, l_id_len:[len(sample)],r:r_3,
                                        image_i:image_1,image_j:image_2})
                #print(XUIJ)
                #print('loss=',_loss)
                #print('auc=',_auc)
                #print(z,ta,b)
                #print('alpha list after softmax:',r3)
                #print('alpha list before softmax:',_a_list)
                train_auc+=_auc
                total_loss+=_loss
                length += 1
            #now1+=1
    
    #print('mine:',xuij_auc/136)    
    #print('a_list_soft:',r3)
    print("total_loss:-----------------", total_loss/length)
    print("train_auc:-------------------", train_auc/length)
    loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
    print('time:',time.time()-t0,' sec')
print('Total cost ',time.time()-t0,' sec')       

Iteraction: 0
total_loss:----------------- [[0.38206159]]
train_auc:------------------- 0.9478372093023256
time: 1437.8446865081787  sec
Iteraction: 1
total_loss:----------------- [[0.28402699]]
train_auc:------------------- 0.9625382059800665
time: 2874.8923552036285  sec
Iteraction: 2
total_loss:----------------- [[0.27727886]]
train_auc:------------------- 0.9660332225913622
time: 4305.701296329498  sec
Iteraction: 3
total_loss:----------------- [[0.27628198]]
train_auc:------------------- 0.9675747508305648
time: 5737.417177200317  sec
Iteraction: 4
total_loss:----------------- [[0.27594401]]
train_auc:------------------- 0.967843853820598
time: 7167.784746646881  sec
Iteraction: 5
total_loss:----------------- [[0.27562736]]
train_auc:------------------- 0.9680797342192691
time: 8600.809064865112  sec
Iteraction: 6
total_loss:----------------- [[0.27545852]]
train_auc:------------------- 0.9685448504983388
time: 10033.134835243225  sec
Iteraction: 7
total_loss:----------------- [[0

In [33]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('time=',loss_acc_list[i][2])

Iteration: 0
loss= [[0.38206159]]
acc= 0.9478372093023256
time= 1437.8446865081787
Iteration: 1
loss= [[0.28402699]]
acc= 0.9625382059800665
time= 2874.8923552036285
Iteration: 2
loss= [[0.27727886]]
acc= 0.9660332225913622
time= 4305.701296329498
Iteration: 3
loss= [[0.27628198]]
acc= 0.9675747508305648
time= 5737.417177200317
Iteration: 4
loss= [[0.27594401]]
acc= 0.967843853820598
time= 7167.784746646881
Iteration: 5
loss= [[0.27562736]]
acc= 0.9680797342192691
time= 8600.809064865112
Iteration: 6
loss= [[0.27545852]]
acc= 0.9685448504983388
time= 10033.134835243225
Iteration: 7
loss= [[0.27549294]]
acc= 0.9681926910299004
time= 11464.157730102539
Iteration: 8
loss= [[0.27507872]]
acc= 0.9688637873754152
time= 12896.273918867111
Iteration: 9
loss= [[0.2751766]]
acc= 0.968843853820598
time= 14326.048789739609


# Get latent factor and Each weight

In [17]:
U, Y, A, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, W1, Wu, Wy, Wa, Wv,Beta])

In [18]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)

User latent shape:  (1489, 128)
photo latent shape:  (88, 128)
Auxilary latent shape:  (88, 128)
W1 weight shape:  (1489, 128)
Wu weight shape: (128, 128)
Wy weight shape: (88, 128, 128)
Wa weight shape: (128, 128)
Wv weight shape: (128, 64)
Beta shape: (1489, 64)


# Testing Part

In [19]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au,np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa,
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av,np.expand_dims(image_2048[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    #print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], image_2048[k].T)
print(RS[s])

0 200
softmax alpha-------------- [0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857]
1 1086
softmax alpha-------------- [0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857]
2 664
softmax alpha-------------- [0.33333333 0.33333333 0.33333333]
3 996
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
4 621
softmax alpha-------------- [0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857]
5 1127
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
6 768
softmax alpha-------------- [0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06

77 749
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
78 82
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
79 475
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
80 975
softmax alpha-------------- [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
81 354
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
82 271
softmax alpha-------------- [0.25 0.25 0.25 0.25]
83 1170
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
84 300
softmax alpha-------------- [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
85 357
softmax alpha-------------- [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
86 875
softmax alpha-------------- [0.33333333 0.33333333 0.33333333]
87 1325
softmax alpha

In [20]:
#取出test的資料
testRS = np.zeros((test_amount,10)) #shape 200*10

#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]


In [40]:
testRS[2]

array([-1.17715875, -1.63641072, -1.14775303, -1.32032336, -1.39914663,
       -1.03070847, -0.7375161 , -1.16313134,  2.11365765, -1.05698223])

In [21]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [41]:
for row in testRS:
    print(row)

[ 1.29576133  0.13842441 -1.19171072 -0.98273902 -0.56716647  0.62553193
  0.60697788 -1.58840041  0.30059072  0.87074044]
[-1.03981251 -1.13951592  1.05676419 -1.39606685  0.04803896 -1.04556422
  0.85760066 -1.12561109 -0.79364071 -1.14592084]
[-1.17715875 -1.63641072 -1.14775303 -1.32032336 -1.39914663 -1.03070847
 -0.7375161  -1.16313134  2.11365765 -1.05698223]
[ 0.69172087 -0.58783957 -1.57980342  0.33066218 -0.09916076  0.30109066
 -1.51337096  0.97485396  0.00554863 -1.74285961]
[ 0.28015029  0.56188546 -0.66813804 -0.50672555 -1.0133413  -1.24134201
  0.23716289  0.66290689 -1.16766556  1.34945722]
[-1.49841606  0.52706248  0.3889642  -1.74082382 -0.23461045 -1.58630319
 -1.58931621 -1.22265928 -1.50652543 -1.64004763]
[ 0.27246899  0.58528437 -0.69474562 -0.1856025  -0.33279677 -0.0441427
  0.59050958 -0.96739257  0.38027705  0.08341256]
[-0.64909497 -0.7604828   1.62903174 -1.09279772 -1.05444281 -1.40618172
 -1.02335052 -1.20313162 -1.54150594 -1.14837105]
[ 1.83228302  2.3

In [23]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),2)
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = len(count_0_all)*len(count_0_all[0])
#print(total) #(200*2)
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        if count_0_all[i][j] < 2: #代表是0或1 (也就是target)
            acc_0 += 1
#print(acc_0)
avg_acc = acc_0/total
print('avg_accuarcy for count_0:',avg_acc)

[0, 9]
[2, 6]
[8, 6]
[7, 0]
[9, 7]
[1, 2]
[6, 1]
[2, 0]
[9, 1]
[3, 0]
[4, 5]
[1, 3]
[3, 8]
[2, 3]
[4, 2]
[0, 1]
[1, 0]
[1, 0]
[1, 0]
[1, 8]
[1, 6]
[0, 7]
[3, 0]
[7, 6]
[1, 0]
[0, 9]
[1, 9]
[6, 1]
[9, 0]
[2, 9]
[0, 5]
[0, 1]
[1, 0]
[0, 5]
[4, 0]
[1, 6]
[3, 8]
[1, 4]
[1, 0]
[0, 1]
[3, 1]
[2, 0]
[8, 4]
[1, 0]
[1, 3]
[9, 5]
[1, 0]
[4, 1]
[2, 6]
[6, 0]
[7, 3]
[7, 0]
[1, 0]
[0, 6]
[4, 5]
[1, 0]
[2, 5]
[6, 5]
[4, 3]
[3, 5]
[2, 0]
[0, 1]
[4, 9]
[7, 4]
[0, 6]
[7, 0]
[0, 1]
[3, 9]
[9, 7]
[7, 3]
[1, 0]
[1, 0]
[1, 0]
[0, 2]
[9, 0]
[7, 1]
[1, 5]
[0, 5]
[0, 2]
[0, 8]
[0, 1]
[9, 6]
[2, 9]
[3, 0]
[7, 0]
[0, 7]
[1, 0]
[2, 1]
[1, 0]
[3, 1]
[0, 4]
[0, 3]
[0, 5]
[8, 2]
[0, 3]
[0, 1]
[0, 1]
[8, 4]
[0, 9]
[7, 4]
[5, 1]
[1, 0]
[0, 9]
[1, 0]
[1, 0]
[1, 6]
[6, 9]
[7, 5]
[0, 1]
[0, 8]
[1, 0]
[4, 6]
[5, 8]
[8, 5]
[1, 9]
[1, 7]
[6, 1]
[0, 1]
[1, 0]
[0, 1]
[0, 4]
[1, 0]
[0, 1]
[8, 9]
[6, 7]
[8, 0]
[7, 9]
[7, 9]
[0, 6]
[1, 0]
[5, 0]
[1, 0]
[8, 1]
[6, 1]
[1, 7]
[1, 0]
[1, 0]
[0, 1]
[5, 1]
[1, 5]
[0, 1]
[0, 1]
[6, 7]

# Top 1 

In [24]:
target = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1)
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] == 1 or top_0[0] == 0:
        target += 1

[0]
[2]
[8]
[7]
[9]
[1]
[6]
[2]
[9]
[3]
[4]
[1]
[3]
[2]
[4]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[3]
[7]
[1]
[0]
[1]
[6]
[9]
[2]
[0]
[0]
[1]
[0]
[4]
[1]
[3]
[1]
[1]
[0]
[3]
[2]
[8]
[1]
[1]
[9]
[1]
[4]
[2]
[6]
[7]
[7]
[1]
[0]
[4]
[1]
[2]
[6]
[4]
[3]
[2]
[0]
[4]
[7]
[0]
[7]
[0]
[3]
[9]
[7]
[1]
[1]
[1]
[0]
[9]
[7]
[1]
[0]
[0]
[0]
[0]
[9]
[2]
[3]
[7]
[0]
[1]
[2]
[1]
[3]
[0]
[0]
[0]
[8]
[0]
[0]
[0]
[8]
[0]
[7]
[5]
[1]
[0]
[1]
[1]
[1]
[6]
[7]
[0]
[0]
[1]
[4]
[5]
[8]
[1]
[1]
[6]
[0]
[1]
[0]
[0]
[1]
[0]
[8]
[6]
[8]
[7]
[7]
[0]
[1]
[5]
[1]
[8]
[6]
[1]
[1]
[1]
[0]
[5]
[1]
[0]
[0]
[6]
[3]
[8]
[1]
[6]
[5]
[0]
[0]


In [25]:
top1_prec = target/len(testRS)
top1_recall = target/(len(testRS)*2)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.52 recall  0.26


# Top 2

In [26]:
target = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),2)
    count_0_all.append(top_0)
    print(top_0)
    for j in range(len(top_0)):
        if top_0[j] == 1 or top_0[j] == 0:
            target += 1

[0, 9]
[2, 6]
[8, 6]
[7, 0]
[9, 7]
[1, 2]
[6, 1]
[2, 0]
[9, 1]
[3, 0]
[4, 5]
[1, 3]
[3, 8]
[2, 3]
[4, 2]
[0, 1]
[1, 0]
[1, 0]
[1, 0]
[1, 8]
[1, 6]
[0, 7]
[3, 0]
[7, 6]
[1, 0]
[0, 9]
[1, 9]
[6, 1]
[9, 0]
[2, 9]
[0, 5]
[0, 1]
[1, 0]
[0, 5]
[4, 0]
[1, 6]
[3, 8]
[1, 4]
[1, 0]
[0, 1]
[3, 1]
[2, 0]
[8, 4]
[1, 0]
[1, 3]
[9, 5]
[1, 0]
[4, 1]
[2, 6]
[6, 0]
[7, 3]
[7, 0]
[1, 0]
[0, 6]
[4, 5]
[1, 0]
[2, 5]
[6, 5]
[4, 3]
[3, 5]
[2, 0]
[0, 1]
[4, 9]
[7, 4]
[0, 6]
[7, 0]
[0, 1]
[3, 9]
[9, 7]
[7, 3]
[1, 0]
[1, 0]
[1, 0]
[0, 2]
[9, 0]
[7, 1]
[1, 5]
[0, 5]
[0, 2]
[0, 8]
[0, 1]
[9, 6]
[2, 9]
[3, 0]
[7, 0]
[0, 7]
[1, 0]
[2, 1]
[1, 0]
[3, 1]
[0, 4]
[0, 3]
[0, 5]
[8, 2]
[0, 3]
[0, 1]
[0, 1]
[8, 4]
[0, 9]
[7, 4]
[5, 1]
[1, 0]
[0, 9]
[1, 0]
[1, 0]
[1, 6]
[6, 9]
[7, 5]
[0, 1]
[0, 8]
[1, 0]
[4, 6]
[5, 8]
[8, 5]
[1, 9]
[1, 7]
[6, 1]
[0, 1]
[1, 0]
[0, 1]
[0, 4]
[1, 0]
[0, 1]
[8, 9]
[6, 7]
[8, 0]
[7, 9]
[7, 9]
[0, 6]
[1, 0]
[5, 0]
[1, 0]
[8, 1]
[6, 1]
[1, 7]
[1, 0]
[1, 0]
[0, 1]
[5, 1]
[1, 5]
[0, 1]
[0, 1]
[6, 7]

In [27]:
top2_prec = target/(len(testRS)*2)
top2_recall = target/(len(testRS)*2)
print('prec ',top2_prec,'recall ',top2_recall)

prec  0.5 recall  0.5


# Top 3

In [28]:
target = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),3)
    count_0_all.append(top_0)
    print(top_0)
    for j in range(len(top_0)):
        if top_0[j] == 1 or top_0[j] == 0:
            target += 1

[0, 9, 5]
[2, 6, 4]
[8, 6, 5]
[7, 0, 3]
[9, 7, 1]
[1, 2, 4]
[6, 1, 8]
[2, 0, 1]
[9, 1, 0]
[3, 0, 1]
[4, 5, 3]
[1, 3, 5]
[3, 8, 5]
[2, 3, 6]
[4, 2, 6]
[0, 1, 4]
[1, 0, 8]
[1, 0, 7]
[1, 0, 3]
[1, 8, 0]
[1, 6, 5]
[0, 7, 1]
[3, 0, 1]
[7, 6, 9]
[1, 0, 7]
[0, 9, 8]
[1, 9, 6]
[6, 1, 0]
[9, 0, 6]
[2, 9, 1]
[0, 5, 9]
[0, 1, 8]
[1, 0, 3]
[0, 5, 7]
[4, 0, 8]
[1, 6, 0]
[3, 8, 6]
[1, 4, 0]
[1, 0, 5]
[0, 1, 8]
[3, 1, 0]
[2, 0, 9]
[8, 4, 0]
[1, 0, 9]
[1, 3, 5]
[9, 5, 1]
[1, 0, 7]
[4, 1, 0]
[2, 6, 7]
[6, 0, 1]
[7, 3, 4]
[7, 0, 9]
[1, 0, 6]
[0, 6, 9]
[4, 5, 7]
[1, 0, 9]
[2, 5, 0]
[6, 5, 1]
[4, 3, 7]
[3, 5, 9]
[2, 0, 7]
[0, 1, 3]
[4, 9, 0]
[7, 4, 6]
[0, 6, 1]
[7, 0, 1]
[0, 1, 4]
[3, 9, 0]
[9, 7, 2]
[7, 3, 1]
[1, 0, 4]
[1, 0, 4]
[1, 0, 6]
[0, 2, 8]
[9, 0, 3]
[7, 1, 8]
[1, 5, 0]
[0, 5, 1]
[0, 2, 3]
[0, 8, 7]
[0, 1, 7]
[9, 6, 1]
[2, 9, 8]
[3, 0, 8]
[7, 0, 9]
[0, 7, 1]
[1, 0, 6]
[2, 1, 0]
[1, 0, 5]
[3, 1, 0]
[0, 4, 1]
[0, 3, 4]
[0, 5, 4]
[8, 2, 0]
[0, 3, 1]
[0, 1, 2]
[0, 1, 4]
[8, 4, 3]
[0, 9, 1]
[7, 4, 1]


In [29]:
top3_prec = target/(len(testRS)*3)
top3_recall = target/(len(testRS)*2)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.44222222222222224 recall  0.6633333333333333
